In [1]:
#Reference for where i got most of the code from https://jrashford.com/2022/01/19/how-to-collect-data-from-reddit-introducing-praw/
import pandas as pd
import praw
from praw.models import MoreComments

In [2]:
reddit = praw.Reddit(client_id='ZeCMY9LXRnsY9M78sS8uGg', client_secret='V-MS7rfVXCJMWLvTfGNyjLLC85ycSg', user_agent='my-app', username='ForThethesis')


In [3]:
subreddit = reddit.subreddit('wallstreetbets')

In [4]:
name = subreddit.display_name
id_ = subreddit.id
public_description = subreddit.public_description
created_at = subreddit.created
print((name, id_, public_description, created_at))

('wallstreetbets', '2th52', 'Like 4chan found a Bloomberg Terminal.', 1328045167.0)


In [22]:
posts = []

In [30]:
for submission in subreddit.hot(limit=1000):
    title = submission.title
    link = submission.url
    author = submission.author_fullname 
    score = submission.score
    created = submission.created    
    print((title, link, author, score, created))
    posts.append(title)

('Weekend Discussion Thread for the Weekend of February 04, 2023', 'https://www.reddit.com/r/wallstreetbets/comments/10sv3v5/weekend_discussion_thread_for_the_weekend_of/', 't2_bd6q5', 159, 1675458008.0)
('When new “expert” regards try to convince veteran regards to get on board their “plays.”', 'https://v.redd.it/eko9udp3txfa1', 't2_3nhqqhl5', 71, 1675396213.0)
('Someone sent me this from some motivational instagram thing...', 'https://i.redd.it/nrs1vtbbpcga1.jpg', 't2_sduosxmu', 4471, 1675576538.0)
('If u ever feel dumb, know that this exists', 'https://i.redd.it/4crqrog4d8ga1.png', 't2_4aw9ufsy', 10218, 1675542025.0)
('Actual footage of bulls next week', 'https://v.redd.it/npkqqdspv8ga1', 't2_qf4g092v', 10141, 1675530296.0)
('This list is tiny... time to update the glossary apes', 'https://i.redd.it/frb1tbudrdga1.jpg', 't2_7xr2zjsi', 142, 1675589339.0)
('A Fellow Regard?', 'https://i.redd.it/xegd1gldmbga1.jpg', 't2_vh1765t6', 503, 1675563442.0)
('when you realized you screwed up', '

In [26]:
df1 = pd.DataFrame({'All The Posts': posts})

In [27]:
df1

,All The Posts
0,Weekend Discussion Thread for the Weekend of F...
1,When new “expert” regards try to convince vete...
2,Someone sent me this from some motivational in...
3,"If u ever feel dumb, know that this exists"
4,Actual footage of bulls next week
...,...
473,Wtf Snap Crash
474,The market right now - how will JPOW change th...
475,WHAT A DAY! JPOW DID WHAT?! SPY BEAT THE DOW?!...
476,$35K 0DTE SPXW Yolo | 295 Contracts | 3960 Str...


In [6]:
body_list = [] 
author_list = []
created_list = []
score_list = []

In [31]:
submission.comments.replace_more(limit=0) # flatten tree
for tl_comment in submission.comments.list():
    body = tl_comment.body
    # author = tl_comment.author.name
    created = tl_comment.created
    score = tl_comment.score
    body_list.append(body)
    # author_list.append(author)
    created_list.append(created)
    score_list.append(score)

In [32]:
# df = pd.DataFrame({'col1': body_list, 'col2': author_list, 'col3': created_list, 'col4': score_list})

In [35]:
df = pd.DataFrame({'Comments': body_list})

In [36]:
df

,Comments
0,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...
1,It’s a RIOT! Wahooo
2,I am very impressed with the performance of HU...
3,Noice
4,"Hey, if you haven't already, please reply to t..."
...,...
472,"No, he was part of a charity organization. Pur..."
473,nice
474,Donating deez nuts.
475,nice


In [37]:
df.to_csv('The_List.csv', )

In [39]:
pd.read_csv(The_List.csv)

NameError: name 'The_List' is not defined

In [42]:
#This transformer model was taking from: https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis?text=Bids+or+offers+include+at+least+1%2C000+shares+and+the+value+of+the+shares+must+correspond+to+at+least+EUR+4%2C000.
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
#              "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
#              "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
#              ]
results = nlp(The_List.csv)
print(results)

[{'label': 'positive', 'score': 0.9998133778572083},
 {'label': 'neutral', 'score': 0.9997822642326355},
 {'label': 'negative', 'score': 0.9877365231513977}]

c:\Users\maart\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
